# Spectra retriever

In [ ]:
import sys, argparse, logging
import os
import pathlib
import glob

import numpy as np
import pandas as pd

from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord

"""
from astroquery.gaia import Gaia
from astroquery.vizier import Vizier
from astroquery.xmatch import XMatch
from astroquery.simbad import Simbad
from astropy.time import Time
"""

#%matplotlib inline


# PLOT STYLE
#plt.style.use('default')
#plt.style.use('seaborn-whitegrid')
# PLOT COLORS
#plt.style.use('seaborn-colorblind') # Color blind friendly palette

: 

# Directories

## Paths

In [3]:
#Main
path_C = '/Users/sebas_astrogsu/Dropbox (GSU Dropbox)/Mac (2)/Documents/RECONS/sebas_recons/RKSTARS/Characterization_Survey/'
path_C_ch = path_C+'CHIRON/'
path_C_tess = path_C+'TESS/'

## Data
path_C_ch_00a = path_C+'CHIRON/00a_TestData/'
path_C_ch_00a_01 = path_C+'CHIRON/00a_TestData/00_RKSTARs/'
path_C_ch_00a_02 = path_C+'CHIRON/00a_TestData/01_HIP003535/'
path_C_ch_00a_03 = path_C+'CHIRON/00a_TestData/02_HIP033560/'
path_C_ch_00a_04 = path_C+'CHIRON/00a_TestData/03_HIP058345/'

path_C_ch_00a_01_data = path_C+'CHIRON/00a_TestData/00_RKSTARs/0_Data_BarCorr/'
path_C_ch_00a_02_data = path_C+'CHIRON/00a_TestData/01_HIP003535/0_Data_BarCorr/'
path_C_ch_00a_03_data = path_C+'CHIRON/00a_TestData/02_HIP033560/0_Data_BarCorr/'
path_C_ch_00a_04_data = path_C+'CHIRON/00a_TestData/03_HIP058345/0_Data_BarCorr/'

path_C_ch_05_00 = path_C+'CHIRON/05_PostRed_EW/00_tsn50spec'

## Codes
path_C_ch_00b = path_C+'CHIRON/00b_CoreCodes/'

## Taks
path_C_ch_01 = path_C+'CHIRON/01_SN_test/'
path_C_ch_02 = path_C+'CHIRON/02_SN_EW/'
path_C_ch_03 = path_C+'CHIRON/03_LiI_zoom/'
path_C_ch_04 = path_C+'CHIRON/04_SpecCorrected/'
path_C_ch_05 = path_C+'CHIRON/05_PostRed_EW/'

## Barycentetr Corrections
path_C_ch_03_BaryCorr = path_C_ch_03 + '0_BaryCorr/'
path_C_ch_03_RVCorr = path_C_ch_03 + '0_RVCorr/'

path_C_ch_05_BaryCorr = path_C_ch_05 + '0_BaryCorr/'
path_C_ch_05_RVCorr = path_C_ch_05 + '0_RVCorr/'

## In Joy

In [ ]:
## Public reduced data
pub_data = 'ftp/pub/SMARTS'

## Already reduced data from pipepline
data_piped = "/nfs/morgan/chiron/tous/mir7/fitspec"

In [4]:
def coping_fits(src, dst,prepdir):
    print('src', src)
    print('dst', dst)
    
    listfits = os.listdir(src)
    listfits.remove('.ipynb_checkpoints')
    #listfits.remove('.ipynb_checkp')
    #print(listfits)
    
    for item in listfits:
     
        creating_dir(prepdir + item[:-5])
        copy_outdir = item[:-5] + '/'
        
        src_file = os.path.join(src, item)
        dst_file = os.path.join(dst,copy_outdir, item)
        print(src_file, dst_file)
        
        if os.path.isdir(src_file):
            coping_fits(src_file)
        else:
            shutil.copy(src_file, dst_file)
            #break

# Masterlist


In [9]:
masterlist = pd.read_csv(path_C+'RKScatalog50pc_all.csv')
masterlist

,RKS,comp,#,type,HIP,G2K,sample1,sample2,RA,DEC,...,pmra,pmdec,Gmag,Bg-Rg,Bg,Rg,PA,DR2_ID,RA_RKS_DD,DEC_RKS_DD
0,RKS0000-1929,-,1,K*,HIP000002,G2K000001,33-40,eq,00:00:01.0,-19:29:55.7,...,163.520,-2.362,8.934,1.235,9.494,8.260,.....,2.341872e+18,0.004167,-19.498806
1,RKS0000-0403,-,1,K*,HIP000018,G2K000002,40-50,eq,00:00:12.7,-04:03:13.2,...,-119.595,24.291,10.441,1.588,11.206,9.618,.....,2.447815e+18,0.052917,-4.053667
2,RKS0000-5650,-,1,K*,HIP000047,NaN,40-50,so,00:00:32.4,-56:50:08.1,...,-44.969,-145.359,10.283,1.428,10.952,9.524,.....,4.922510e+18,0.135000,-56.835583
3,RKS0000+1829,-,1,K*,NaN,G2K000003,33-40,eq,00:00:39.3,+18:29:19.8,...,334.563,196.005,10.412,1.698,11.244,9.547,.....,2.774087e+18,0.163750,18.488833
4,RKS0000-6940,-,1,K*,HIP000057,NaN,00-33,so,00:00:40.3,-69:40:33.4,...,-40.569,-67.540,8.014,1.043,8.477,7.434,.....,4.703237e+18,0.167917,-69.675944
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5177,RKS2359+5906,-,1,K*,NaN,NaN,40-50,no,23:59:12.5,+59:06:22.2,...,114.739,70.425,9.530,1.146,10.044,8.898,.....,1.999241e+18,359.802083,59.106167
5178,RKS2359-2602,-,1,K*,HIP118261,G2K002553,00-33,eq,23:59:13.7,-26:02:55.4,...,-229.719,12.321,8.336,1.283,8.928,7.645,.....,2.334879e+18,359.807083,-26.048722
5179,RKS2359-1656,-,1,K*,NaN,G2K002554,33-40,eq,23:59:27.9,-16:56:40.9,...,1128.658,-49.646,8.698,0.988,9.127,8.140,.....,2.415244e+18,359.866250,-16.944694
5180,RKS2359+0639,-,1,K*,HIP118310,G2K002555,00-33,eq,23:59:47.8,+06:39:50.9,...,-58.878,-166.649,8.491,1.406,9.153,7.747,.....,2.745473e+18,359.949167,6.664139


In [ ]:
from utils.terminal_log import (log,
                                degree_to_dms,
                                degree_to_hms,
                                degree_to_mas)

# Retriever

In [ ]:
from utils.terminal_log import (log,
                                degree_to_dms,
                                degree_to_hms,
                                degree_to_mas)

# Funtion: select sepctra with OBJECT id from RKSTAR Masterlist
def RKS_spec(dataset_root,out_name):
    
    ## Public reduced data
    pub_data = 'ftp/pub/SMARTS'
    masterlist = pd.read_csv(path_C+'RKScatalog50pc_all.csv')

    # dataset to match
    datasets = [dI for dI in os.listdir(dataset_root) if os.path.isdir(os.path.join(dataset_root,dI))]
    log.info('Datasets retrieved')

In [ ]:
a=1